##  CNN做MNIST分类

使用 tf.layers 高级 API 构建 CNN 

In [1]:
import numpy as np
import tensorflow as tf

# 设置按需使用GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

import time

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1.导入数据，用 tensorflow 导入

In [2]:
# 用tensorflow 导入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)
# 看看咱们样本的数量
print(mnist.test.labels.shape)
print(mnist.train.labels.shape)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
(10000,)
(55000,)


## 2. 构建网络

In [3]:
with tf.name_scope('inputs'):
    X_ = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.int64, [None])

# 把X转为卷积所需要的形式
X = tf.reshape(X_, [-1, 28, 28, 1])
h_conv1 = tf.layers.conv2d(X, filters=32, kernel_size=5, strides=1, padding='same', activation=tf.nn.relu, name='conv1')
h_pool1 = tf.layers.max_pooling2d(h_conv1, pool_size=2, strides=2, padding='same', name='pool1')

h_conv2 = tf.layers.conv2d(h_pool1, filters=64, kernel_size=5, strides=1, padding='same',activation=tf.nn.relu, name='conv2')
h_pool2 = tf.layers.max_pooling2d(h_conv2, pool_size=2, strides=2, padding='same', name='pool2')

# flatten
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.layers.dense(h_pool2_flat, 1024, name='fc1', activation=tf.nn.relu)

# dropout: 输出的维度和h_fc1一样，只是随机部分值被值为零
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, 0.5)   # 实际测试的时候这里不应该使用 0.5，这里为了方便演示都这样写而已
h_fc2 = tf.layers.dense(h_fc1_drop, units=10, name='fc2')
# y_conv = tf.nn.softmax(h_fc2)
y_conv = h_fc2
print('Finished building network.')

Finished building network.


In [4]:
print(h_conv1)
print(h_pool1)
print(h_conv2)
print(h_pool2)

print(h_pool2_flat)
print(h_fc1)
print(h_fc2)

Tensor("conv1/Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("pool1/MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("conv2/Relu:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("pool2/MaxPool:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
Tensor("fc1/Relu:0", shape=(?, 1024), dtype=float32)
Tensor("fc2/BiasAdd:0", shape=(?, 10), dtype=float32)


## 3.训练和评估

<b> 在测试的时候不使用 mini_batch， 那么测试的时候会占用较多的GPU（4497M），这在 notebook 交互式编程中是不推荐的。

In [5]:
# cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.cast(y_, dtype=tf.int32), logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())

tic = time.time()
for i in range(10000):
    batch = mnist.train.next_batch(100)
    if i%1000 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            X_:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step {}, training accuracy = {:.4f}, pass {:.2f}s ".format(i, train_accuracy, time.time() - tic))
    train_step.run(feed_dict={X_: batch[0], y_: batch[1]})

print("test accuracy %g"%accuracy.eval(feed_dict={
    X_: mnist.test.images, y_: mnist.test.labels}))

step 0, training accuracy = 0.1100, pass 1.18s 
step 1000, training accuracy = 0.9800, pass 6.01s 
step 2000, training accuracy = 0.9800, pass 10.82s 
step 3000, training accuracy = 0.9700, pass 15.65s 
step 4000, training accuracy = 1.0000, pass 20.40s 
step 5000, training accuracy = 0.9800, pass 25.26s 
step 6000, training accuracy = 1.0000, pass 30.00s 
step 7000, training accuracy = 0.9900, pass 34.84s 
step 8000, training accuracy = 1.0000, pass 39.66s 
step 9000, training accuracy = 1.0000, pass 44.55s 
test accuracy 0.9924
